In [1]:
data_var = '2023-12-23'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10059,2023-12-23,Brasil Nbb,11:00,Brasília,São Paulo,3.69,1.25,160.5,1.86,1.84,7.5,2.06,1.68,rmKzyCR1,0.271003,0.800000,0.537634,0.543478,0.071003,0.6,0.6,NaN,NaN,464.468,253.893632,0.546633,114.448,40.466944,0.353584,175.266,121.158,249.09,99.90,0.0,0.0,0.0,0.0,0.698518,0.007644,0.143690,-2.81,-0.562,-4.786477,0.0,0.0,0.0,-0.75,-0.150,-1.666667,0.0,0.0,0.0
10060,2023-12-23,Brasil Nbb,11:00,R10 Score Vasco da Gama,Fortaleza B.C.,1.44,2.65,147.5,1.88,1.79,-6.5,2.09,1.66,4dLWyhCe,0.694444,0.377358,0.531915,0.558659,0.071803,0.2,0.6,NaN,NaN,176.784,44.723626,0.252985,200.908,134.962348,0.671762,132.928,266.996,153.90,352.24,0.0,0.0,0.0,0.0,0.418386,0.034681,0.162163,1.11,0.222,1.981982,0.0,0.0,0.0,0.97,0.194,8.505155,0.0,0.0,0.0
10061,2023-12-23,Brasil Nbb,17:00,Flamengo,Unifacisa,1.19,4.37,155.5,1.84,1.86,-11.5,1.97,1.75,dj8Iae42,0.840336,0.228833,0.543478,0.537634,0.069169,0.4,0.6,NaN,NaN,119.158,26.945149,0.226130,117.088,15.705379,0.134133,109.238,157.226,89.32,137.28,0.0,0.0,0.0,0.0,0.808849,0.007644,0.083636,1.18,0.236,0.805085,0.0,0.0,0.0,-2.18,-0.436,-7.729358,0.0,0.0,0.0
10062,2023-12-23,Brasil Nbb,17:00,São José,Franca,2.56,1.47,161.5,1.85,1.85,3.5,2.10,1.66,AgCMbFJ8,0.390625,0.680272,0.540541,0.540541,0.070897,0.8,0.2,NaN,NaN,112.094,11.520659,0.102777,100.640,20.611448,0.204804,156.182,116.130,107.30,82.50,0.0,0.0,0.0,0.0,0.382504,0.000000,0.165493,2.96,0.592,2.635135,0.0,0.0,0.0,-0.46,-0.092,-5.108696,0.0,0.0,0.0
10063,2023-12-23,Brasil Nbb,17:00,União Corinthians,Pinheiros,2.12,1.67,154.5,1.81,1.86,1.5,1.96,1.75,hI1RcZYE,0.471698,0.598802,0.552486,0.537634,0.070501,0.2,0.8,NaN,NaN,247.710,72.655561,0.293309,144.964,32.290563,0.222749,173.018,234.288,207.46,106.88,0.0,0.0,0.0,0.0,0.167915,0.019267,0.080050,-3.63,-0.726,-1.542700,0.0,0.0,0.0,-0.13,-0.026,-25.769231,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10291,2023-12-23,Turquia Kbsl Feminina,07:30,Botas F,Cankaya F,1.63,2.19,161.5,1.81,1.85,-3.5,2.00,1.72,4E6ueMUC,0.613497,0.456621,0.552486,0.540541,0.070118,0.0,0.0,NaN,NaN,128.926,25.161407,0.195162,426.626,444.755888,1.042496,0.000,0.000,150.23,95.55,0.0,0.0,0.0,0.0,0.207319,0.015456,0.106446,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
10292,2023-12-23,Turquia Kbsl Feminina,09:00,Fenerbahce F,CBK Mersin F,1.19,4.27,156.5,1.87,1.79,-11.5,2.09,1.66,IHAqftpJ,0.840336,0.234192,0.534759,0.558659,0.074528,0.0,0.0,NaN,NaN,NaN,NaN,NaN,108.462,15.504305,0.142947,0.000,0.000,0.00,122.84,0.0,0.0,0.0,0.0,0.797761,0.030912,0.162163,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
10293,2023-12-23,Turquia Kbsl Feminina,11:00,Emlak Konut F,Galatasaray F,2.53,1.48,153.5,1.88,1.79,3.5,1.95,1.76,KMlSIPMJ,0.395257,0.675676,0.531915,0.558659,0.070933,0.0,0.0,NaN,NaN,193.934,87.903707,0.453266,127.470,39.031279,0.306200,0.000,0.000,336.53,147.25,1.0,0.0,0.0,0.0,0.370305,0.034681,0.072426,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
10294,2023-12-23,Turquia Tkbl Feminina,08:00,Yalova F,Burhaniye F,1.21,4.12,141.5,1.83,1.83,-10.5,1.96,1.75,jBf7BNcl,0.826446,0.242718,0.546448,0.546448,0.069165,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,243.54,486.75,0.0,0.0,0.0,0.0,0.772113,0.000000,0.080050,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
8,00:00,Eua Nba,Golden State Warriors,Washington Wizards,246.5,1.89,1.0000,Over
9,00:00,Eua Nba,Sacramento Kings,Phoenix Suns,243.5,1.89,1.0000,Over
10,14:30,Eua Nba,New York Knicks,Milwaukee Bucks,238.5,1.81,0.9999,Over
12,21:00,Eua Nba,Charlotte Hornets,Denver Nuggets,225.5,1.86,1.0000,Over
13,21:00,Eua Nba,Indiana Pacers,Orlando Magic,241.5,1.84,1.0000,Over
14,21:30,Eua Nba,Atlanta Hawks,Memphis Grizzlies,238.5,1.81,1.0000,Over
15,21:30,Eua Nba,Brooklyn Nets,Detroit Pistons,231.5,1.83,0.9999,Over
16,21:30,Eua Nba,Toronto Raptors,Utah Jazz,228.5,1.88,1.0000,Over
